In [1]:
import pandas as pd
import numpy as np

In [2]:
train_txt = pd.read_csv('ratings_train.txt',sep='\t')
train_txt

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [3]:
from konlpy.tag import *

In [4]:
train_txt = train_txt.dropna().reset_index(drop=True)

In [5]:
train_txt['document'] = train_txt['document'].apply(lambda x:x.replace('.',' '))

In [6]:
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
mecab = Mecab()
okt = Okt()

In [7]:
from collections import defaultdict
def get_ngram_counter(docs, min_count=10, n_range=(1,1)):

    def to_ngrams(words, n):
        ngrams = []
        for b in range(0, len(words) - n + 1):
            ngrams.append(tuple(words[b:b+n]))
        return ngrams

    n_begin, n_end = n_range
    ngram_counter = defaultdict(int)
    for doc in docs:
        words = mecab.pos(doc, join=True)
        for n in range(n_begin, n_end + 1):
            for ngram in to_ngrams(words, n):
                ngram_counter[ngram] += 1

    ngram_counter = {
        ngram:count for ngram, count in ngram_counter.items()
        if count >= min_count
    }

    return ngram_counter

ngram_counter = get_ngram_counter(train_txt.document)

In [8]:
class NgramTokenizer:

    def __init__(self, ngrams, base_tokenizer, n_range=(1, 1)):
        self.ngrams = ngrams
        self.base_tokenizer = base_tokenizer
        self.n_range = n_range

    def __call__(self, sent):
        return self.tokenize(sent)

    def tokenize(self, sent):
        if not sent:
            return []

        unigrams = self.base_tokenizer.pos(sent, join=True)

        n_begin, n_end = self.n_range
        ngrams = []
        for n in range(n_begin, n_end + 1):
            for ngram in self._to_ngrams(unigrams, n):
                ngrams.append('-'.join(ngram))
        return ngrams

    def _to_ngrams(self, words, n):
        ngrams = []
        for b in range(0, len(words) - n + 1):
            ngram = tuple(words[b:b+n])
            if ngram in self.ngrams:
                ngrams.append(ngram)
        return ngrams

ngram_tokenizer = NgramTokenizer(ngram_counter, mecab)

In [9]:
from tqdm import tqdm

token_list = []
for doc in tqdm(train_txt.document):
    token_list.append(ngram_tokenizer(doc))

100%|██████████| 149995/149995 [00:16<00:00, 9298.09it/s] 


In [10]:
train_txt['token'] = token_list
train_txt

,id,document,label,token
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,"[아/IC, 더/MAG, 빙/MAG, 진짜/MAG, 짜증/NNG, 나/NP, 네요/..."
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,"[흠/IC, 포스터/NNP, 보고/JKB, 초딩/NNG, 영화/NNG, 줄/NNG,..."
2,10265843,너무재밓었다그래서보는것을추천한다,0,"[너무/MAG, 재/XPN]"
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,"[교도소/NNG, 이야기/NNG, 솔직히/MAG, 재미/NNG, 는/JX, 없/VA..."
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"[의/JKG, 익살/NNG, 스런/XSA+ETM, 연기/NNG, 가/JKS, 돋보였..."
...,...,...,...,...
149990,6222902,인간이 문제지 소는 뭔죄인가,0,"[인간/NNG, 이/JKS, 문제지/NNG, 소/NNG, 는/JX, 뭔/MM, 죄/..."
149991,8549745,평점이 너무 낮아서,1,"[평점/NNG, 이/JKS, 너무/MAG, 낮/VA, 아서/EC]"
149992,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0,"[이게/NP+JKS, ?/SF, 한국인/NNG, 은/JX, 고/EC, 은/JX, 착..."
149993,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1,"[청춘/NNG, 영화/NNG, 의/JKG, 최고봉/NNP, 방황/NNG, 과/JC,..."


In [11]:
for item in train_txt['token'].tolist():
    if len(item) != 0:
        item = ''.join(str(e)+',' for e in item)
        item = item.replace('[','').replace(']','')
    else:
        item = ''

In [12]:
for i in range(len(train_txt['token'].tolist())):
    if len(item) != 0:
        train_txt['token'][i] = ''.join(str(e)+',' for e in train_txt['token'][i])
        train_txt['token'][i] = train_txt['token'][i].replace('[','').replace(']','')
    else:
        train_txt['token'][i] = ''

<ipython-input-12-0bd6096556dd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_txt['token'][i] = ''.join(str(e)+',' for e in train_txt['token'][i])
<ipython-input-12-0bd6096556dd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_txt['token'][i] = train_txt['token'][i].replace('[','').replace(']','')


In [13]:
train_txt['tokens_'] = train_txt['token'].apply(lambda x: ' '.join([w.split('/')[0] for w in x.split(',')]).replace('[','').replace("'",""))

In [14]:
train_txt

,id,document,label,token,tokens_
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,"아/IC,더/MAG,빙/MAG,진짜/MAG,짜증/NNG,나/NP,네요/VCP+EC,...",아 더 빙 진짜 짜증 나 네요 목소리
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,"흠/IC,포스터/NNP,보고/JKB,초딩/NNG,영화/NNG,줄/NNG,오버/NNG...",흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍 지 않 구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,"너무/MAG,재/XPN,",너무 재
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,"교도소/NNG,이야기/NNG,솔직히/MAG,재미/NNG,는/JX,없/VA,다/EC,...",교도소 이야기 솔직히 재미 는 없 다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"의/JKG,익살/NNG,스런/XSA+ETM,연기/NNG,가/JKS,돋보였/VV+EP...",의 익살 스런 연기 가 돋보였 던 영화 ! 스파이더맨 에서 늙 어 보이 기 만 했 ...
...,...,...,...,...,...
149990,6222902,인간이 문제지 소는 뭔죄인가,0,"인간/NNG,이/JKS,문제지/NNG,소/NNG,는/JX,뭔/MM,죄/NNG,인가/...",인간 이 문제지 소 는 뭔 죄 인가
149991,8549745,평점이 너무 낮아서,1,"평점/NNG,이/JKS,너무/MAG,낮/VA,아서/EC,",평점 이 너무 낮 아서
149992,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0,"이게/NP+JKS,?/SF,한국인/NNG,은/JX,고/EC,은/JX,착하/VA,다/...",이게 ? 한국인 은 고 은 착하 다 ?
149993,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1,"청춘/NNG,영화/NNG,의/JKG,최고봉/NNP,방황/NNG,과/JC,우울/NNG...",청춘 영화 의 최고봉 방황 과 우울 했 던 날 들 의 자화상


In [68]:
train_txt.to_pickle('train_mecab_df.pkl')

In [69]:
train_df = pd.read_pickle('train_mecab_df.pkl')
train_df

,id,document,label,token,tokens_
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,"아/IC,더/MAG,빙/MAG,진짜/MAG,짜증/NNG,나/NP,네요/VCP+EC,...",아 더 빙 진짜 짜증 나 네요 목소리
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,"흠/IC,포스터/NNP,보고/JKB,초딩/NNG,영화/NNG,줄/NNG,오버/NNG...",흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍 지 않 구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,"너무/MAG,재/XPN,",너무 재
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,"교도소/NNG,이야기/NNG,솔직히/MAG,재미/NNG,는/JX,없/VA,다/EC,...",교도소 이야기 솔직히 재미 는 없 다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"의/JKG,익살/NNG,스런/XSA+ETM,연기/NNG,가/JKS,돋보였/VV+EP...",의 익살 스런 연기 가 돋보였 던 영화 ! 스파이더맨 에서 늙 어 보이 기 만 했 ...
...,...,...,...,...,...
149990,6222902,인간이 문제지 소는 뭔죄인가,0,"인간/NNG,이/JKS,문제지/NNG,소/NNG,는/JX,뭔/MM,죄/NNG,인가/...",인간 이 문제지 소 는 뭔 죄 인가
149991,8549745,평점이 너무 낮아서,1,"평점/NNG,이/JKS,너무/MAG,낮/VA,아서/EC,",평점 이 너무 낮 아서
149992,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0,"이게/NP+JKS,?/SF,한국인/NNG,은/JX,고/EC,은/JX,착하/VA,다/...",이게 ? 한국인 은 고 은 착하 다 ?
149993,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1,"청춘/NNG,영화/NNG,의/JKG,최고봉/NNP,방황/NNG,과/JC,우울/NNG...",청춘 영화 의 최고봉 방황 과 우울 했 던 날 들 의 자화상


### 불용어 제거<br>
[한국어 불용어 사전](https://www.ranks.nl/stopwords/korean)

In [23]:
stopwords_df = pd.read_excel('stopwords.xlsx')
stopwords_df

,불용어
0,아
1,휴
2,아이구
3,아이쿠
4,아이고
...,...
670,일곱
671,여덟
672,아홉
673,령


In [24]:
stopwords_list = stopwords_df['불용어'].to_list()
stopwords_list

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

In [27]:
tokens_list = [tokens.split() for tokens in train_txt['tokens_'].tolist() if tokens not in stopwords_list]

### Word2Vec 모델 생성

In [28]:
from gensim.models import Word2Vec

word2vec = Word2Vec(
    sentences = tokens_list,
    size = 300,
    min_count = 2,
    window = 10,
    sg = 1,
    hs = 1,
    iter = 10
)

In [29]:
word2vec.save('NaverMovie_mecab300_removal_sw.model')

In [30]:
tokens_list

[['아', '더', '빙', '진짜', '짜증', '나', '네요', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍', '지', '않', '구나'],
 ['너무', '재'],
 ['교도소', '이야기', '솔직히', '재미', '는', '없', '다', '평점', '조정'],
 ['의',
  '익살',
  '스런',
  '연기',
  '가',
  '돋보였',
  '던',
  '영화',
  '!',
  '스파이더맨',
  '에서',
  '늙',
  '어',
  '보이',
  '기',
  '만',
  '했',
  '던',
  '가',
  '너무나',
  '도',
  '이뻐',
  '보였',
  '다'],
 ['막',
  '3',
  '세',
  '부터',
  '초등',
  '학교',
  '1',
  '학년',
  '생',
  '인',
  '8',
  '살',
  '용',
  '영화',
  'ㅋㅋㅋ',
  '별반',
  '개',
  '도',
  '아까움'],
 ['원작', '의', '긴장감', '을', '제대로', '살려', '내', '지', '못했', '다'],
 ['별',
  '반개',
  '도',
  '아깝',
  '다',
  '욕',
  '나온다',
  '연기',
  '생활',
  '이',
  '몇',
  '년',
  '인지',
  '정말',
  '발',
  '로',
  '해도',
  '그것',
  '보단',
  '낫',
  '겟',
  '다',
  '납치',
  '감금',
  '만',
  '반복',
  '반복',
  '이',
  '드라마',
  '는',
  '가족',
  '도',
  '없',
  '다',
  '연기',
  '못',
  '하',
  '는',
  '사람',
  '만',
  '네'],
 ['액션', '이', '없', '는데', '도', '재미', '있', '는', '몇', '안', '되', '는', '영화'],
 ['왜',
  '케',
  '평점',
  '이',
  '낮'